# Beam Lab Code ＼(^o^)／

## Beam dimensions and material properties

In [ ]:
# 3-points beam bending
L = 0.700
W = 0.040
H = 0.025
I = W*H**3/12

# Material Properties
E = 69000e6  # Elastic Modulus
nu = 0.33     # Poisson's Ratio
Y = 275e6    # Yield Strength


## Gauge positions and orientations

In [ ]:
from math import pi

#Gauge 1
G1x = 0.350
G1z = -H/2
th1a = 0
th1b = pi/4
th1c = pi/2

#Gauge 2
G2x = 0.375
G2z = -H/2 + 0.0125
th2a = 0
th2b = pi/4
th2c = pi/2

#Gauge 3
G3x = 0.350
G3z = H/2
th3a = 0
th3b = pi/4
th3c = pi/2

#Gauge 4
G4x = 0.450
G4z = -H/2
th4a = -pi/4
th4b = 0
th4c = pi/4

#3-Point Load Point
XLoad = 0.350

#4-Point Load Points
a = 0.200
b = 0.500
c = 0.150

## Calculate the max deflection

In [ ]:
# 3-point beam
def beam3Deflection(P,E,I,L):
    return P*L**3/(48*E*I)

# 4-point beam
def beam4Deflection(P,E,I,L,a):
    return P*a*(3*L**2-4*a**2)/(48*E*I)

#Example
#Note: this uses properties and dimensions from cells above
load = 100
deflect3 = beam3Deflection(load,E,I,L)
deflect4 = beam4Deflection(load,E,I,L,a)
#Print values
print(deflect3)
print(deflect4)
#To print out in mm rounded to the nearest 0.01
print(round(deflect3*1e3,2))
print(round(deflect4*1e3,2))

## Calculate the theoretical strain value

In [ ]:
def beam3Strain(P,E,nu,I,L,H,x,z):
    #Calculate shear and moment
    if x < L/2:
        Q = P/2
        M = P*x/2
    else:
        Q = -P/2
        M = P*(L-x)/2

    #Calculate stress
    sigmaX = M*z/I
    tau = Q/2/I*((H/2)**2 - z**2)

    #Calculate strain
    epsilonX = sigmaX/E
    epsilonY = -sigmaX*nu/E
    gamma = tau*2*(1+nu)/E #tau/G - G=E/2/(1+nu)

    return [epsilonX, epsilonY, gamma]

def beam4Strain(P,E,nu,I,L,H,a,x,z):
    #Calculate shear and moment
    if x < a:
        Q = P/2
        M = P*x/2
    elif x < L-a:
        Q = 0
        M = P*a/2
    else:
        Q = -P/2
        M = P*(L-x)/2

    #Calculate stress
    sigmaX = M*z/I
    tau = Q/2/I*((H/2)**2 - z**2)

    #Calculate strain
    epsilonX = sigmaX/E
    epsilonY = -sigmaX*nu/E
    gamma = tau*2*(1+nu)/E #tau/G - G=E/2/(1+nu)

    return [epsilonX, epsilonY, gamma]

#Example
#Note: this uses properties and dimensions from cells above
load = 100
xPos = 0.350
zPos = -H/4
strain3 = beam3Strain(load,E,nu,I,L,H,xPos,zPos)
strain4 = beam4Strain(load,E,nu,I,L,H,a,xPos,zPos)

#Print values
print(strain3)
print(strain4)
#To print out microstrain rounded to the nearest 0.1
print([round(x*1e6,1) for x in strain3])
print([round(x*1e6,1) for x in strain4])

## Rotate the strain gauge reading

In [ ]:
from math import cos, sin #note: angles should be in radians

def rotateStrain(ex, ey, exy, th):
    exp = (ex+ey)/2 + (ex-ey)/2*cos(2*th) + exy*sin(2*th)
    eyp = (ex+ey)/2 - (ex-ey)/2*cos(2*th) - exy*sin(2*th)
    exyp = exy*cos(2*th) - (ex-ey)/2*sin(2*th)
    return [exp, eyp, exyp]

#Example
#Note: this uses properties and dimensions from cells above
thG = pi/4
eGx, eGy, eGxy = beam3Strain(load,E,nu,I,L,H,xPos,zPos)
strainG = rotateStrain(eGx, eGy, eGxy, thG)

#Print values
print(strainG)
#To print out microstrain rounded to the nearest 0.1
print([round(x*1e6,1) for x in strainG])

## Plot the deflection and strain gauge reading

In [ ]:
import matplotlib.pyplot as plt

#Experimental displacement and strains
Loads = [0, 200, 600]
eDeflect = [0, 0.9, 2.6]
eGauge1a = [0, -235, -700]

#Theoretical displacement and strains
thDeflect = []
thGauge1a = []
for load in Loads:
    thDeflect += [beam3Deflection(2*load,E,I,L)*1e3]
    eGx, eGy, eGxy = beam3Strain(2*load,E,nu,I,L,H,G1x,G1z)
    print(eGx, eGy, eGxy)
    thGauge1a += [rotateStrain(eGx, eGy, eGxy, th1a)[0]*1e6]

#Plot the results
fig,axs = plt.subplots(1,2)
fig.suptitle("3-Point Beam", fontsize=16)

#Plot deflection
axs[0].plot(Loads, eDeflect, '.-', label='Experiment')
axs[0].plot(Loads, thDeflect, '.-', label='Theory')
axs[0].set_xlabel('Load(N)')
axs[0].set_ylabel('Deflection(mm)')
axs[0].title.set_text('Deflection')
axs[0].legend()

#Plot deflection
axs[1].plot(Loads, eGauge1a, '.-', label='Experiment')
axs[1].plot(Loads, thGauge1a, '.-', label='Theory')
axs[1].set_xlabel('Load(N)')
axs[1].set_ylabel('Strain (microstrain)')
axs[1].title.set_text('Strain Gauge 1a')
axs[1].legend()

plt.tight_layout()